In [1]:
import torch

In [2]:
N = 20
M = 10

In [3]:
W1 = torch.randn([N,M])
W2 = torch.randn([N,M])

In [4]:
A_pre = torch.nn.Parameter(torch.eye(N))
A_post = torch.nn.Parameter(torch.eye(M))

### A_pre

In [5]:
optimiser = torch.optim.Adam([A_pre], lr = 1e-3)
criteria = torch.nn.MSELoss()

In [6]:
for i in range(10000):
    optimiser.zero_grad()
    loss = criteria(W2, A_pre @ W1)
    loss.backward()
    optimiser.step()
    if i%1000==0:
        print(loss)

tensor(1.9945, grad_fn=<MseLossBackward0>)
tensor(0.0050, grad_fn=<MseLossBackward0>)
tensor(1.4378e-06, grad_fn=<MseLossBackward0>)
tensor(7.2493e-12, grad_fn=<MseLossBackward0>)
tensor(3.8200e-11, grad_fn=<MseLossBackward0>)
tensor(1.4240e-10, grad_fn=<MseLossBackward0>)
tensor(2.9684e-10, grad_fn=<MseLossBackward0>)
tensor(2.0537e-09, grad_fn=<MseLossBackward0>)
tensor(1.8333e-11, grad_fn=<MseLossBackward0>)
tensor(1.8016e-09, grad_fn=<MseLossBackward0>)


### A_post

In [7]:
optimiser = torch.optim.Adam([A_post], lr = 1e-3)
criteria = torch.nn.MSELoss()

In [8]:
for i in range(10000):
    optimiser.zero_grad()
    loss = criteria(W2, W1 @ A_post)
    loss.backward()
    optimiser.step()
    if i%1000==0:
        print(loss)

tensor(1.9945, grad_fn=<MseLossBackward0>)
tensor(0.5985, grad_fn=<MseLossBackward0>)
tensor(0.4943, grad_fn=<MseLossBackward0>)
tensor(0.4883, grad_fn=<MseLossBackward0>)
tensor(0.4882, grad_fn=<MseLossBackward0>)
tensor(0.4882, grad_fn=<MseLossBackward0>)
tensor(0.4882, grad_fn=<MseLossBackward0>)
tensor(0.4882, grad_fn=<MseLossBackward0>)
tensor(0.4882, grad_fn=<MseLossBackward0>)
tensor(0.4882, grad_fn=<MseLossBackward0>)


In [9]:
def init_lora_matrices(A, B):
    N,R = A.shape
    A.requires_grad = False
    B.requires_grad = False
    B[0][0] = 1
    A[:,0] = 1
    A.requires_grad = True
    B.requires_grad = True

In [10]:
def shift(M):
    n_rows, n_cols = M.shape
    arange2 = torch.arange(n_cols).view(( 1,n_cols)).repeat((n_rows,1))
    for i in range(n_rows):
        for j in range(n_cols):
            arange2[i][j] = ((i - j) % n_cols)
    return torch.gather(M, 1, arange2)

### A_pre_r

In [11]:
R = 1
while R <= N:
    print("#"*5 + " Rank Post " + str(R) + " " + "#"*5)
    A1 = torch.nn.Parameter(torch.randn([N,R]))
    B1 = torch.nn.Parameter(torch.zeros([R,N]))
    init_lora_matrices(A1,B1)
    optimiser = torch.optim.Adam([A1, B1], lr = 1e-3)
    criteria = torch.nn.MSELoss()
    for i in range(10000):
        optimiser.zero_grad()
        loss = criteria(W2, shift(A1 @ B1) @ W1)
        loss.backward()
        optimiser.step()
    print(loss)
    R=R*2

##### Rank Post 1 #####
tensor(0.6557, grad_fn=<MseLossBackward0>)
##### Rank Post 2 #####
tensor(0.3853, grad_fn=<MseLossBackward0>)
##### Rank Post 4 #####
tensor(0.1144, grad_fn=<MseLossBackward0>)
##### Rank Post 8 #####
tensor(1.7161e-07, grad_fn=<MseLossBackward0>)
##### Rank Post 16 #####
tensor(4.8441e-07, grad_fn=<MseLossBackward0>)


### A_post_r

In [12]:
R = 1
while R <= M:
    print("#"*5 + " Rank Pre " + str(R) + " " + "#"*5)
    A1 = torch.nn.Parameter(torch.randn([M,R]))
    B1 = torch.nn.Parameter(torch.zeros([R,M]))
    init_lora_matrices(A1,B1)
    optimiser = torch.optim.Adam([A1, B1], lr = 1e-3)
    criteria = torch.nn.MSELoss()
    for i in range(10000):
        optimiser.zero_grad()
        loss = criteria(W2, W1 @ shift(A1 @ B1))
        loss.backward()
        optimiser.step()
    print(loss)
    R=R*2

##### Rank Pre 1 #####
tensor(0.8304, grad_fn=<MseLossBackward0>)
##### Rank Pre 2 #####
tensor(0.6946, grad_fn=<MseLossBackward0>)
##### Rank Pre 4 #####
tensor(0.5611, grad_fn=<MseLossBackward0>)
##### Rank Pre 8 #####
tensor(0.4909, grad_fn=<MseLossBackward0>)
